In [1]:
#import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read tracking and player_play csvs
tracking_football = pd.read_csv('tracking_football.csv')
tracking_simple = pd.read_csv('tracking_simple.csv')
df_player_play_motion = pd.read_csv('player_play.csv')
df_player = pd.read_csv('players.csv')

In [3]:
#this function adds a column from one DataFrame (other_df) to another DataFrame (df) based on specified merge keys. 
#it allows for renaming the added column in the resulting DataFrame.
def add_column_from_other_df(df, other_df, col, new_col_name, merge_keys=['gameId', 'playId', 'nflId']):

    #shrink other dataframe
    df_small = other_df[merge_keys + [col]]
    
    #put placeholder column name that won't be in df
    df_small.rename(columns={col: 'placeholder_column_x09ds623n'}, inplace=True)
    
    #merge the dataframes
    df = df.merge(df_small, on = merge_keys)

    #rename the new column
    df.rename(columns={'placeholder_column_x09ds623n': new_col_name}, inplace=True)

    return df


In [4]:
#get the frames of ball snap and the last 'in_motion' frame
last_motion_frame = tracking_simple[tracking_simple['in_motion'] == 1].groupby(['gameId', 'playId', 'nflId'])['frameId'].max().sub(1).reset_index()
#note:subtracted 1 above to go to the frame before ball snap, as some players not in motion moved exactly at the snap and this gives a better buffer
snap_frame = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['frameId'].max().reset_index()

df_player_play_motion = add_column_from_other_df(df_player_play_motion, last_motion_frame, 'frameId', 'last_motion_frameId')
df_player_play_motion = add_column_from_other_df(df_player_play_motion, snap_frame, 'frameId', 'snap_frameId')


In [5]:
#find frame gap between the snap and the last 'in_motion' frame
df_player_play_motion['snap_motion_gap'] = df_player_play_motion['snap_frameId'] - df_player_play_motion['last_motion_frameId'] - 1 #account for adjustment above

In [6]:
# #lines up with inMotionAtBallSnap
# df_player_play_motion[['inMotionAtBallSnap','snap_motion_gap']].head()

In [7]:
tracking_simple.columns

Index(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'club', 'event',
       'x_std', 'y_std', 'o_std', 'dir_std', 's_std', 'a_std', 'dis_std',
       'x_std_fb', 'y_std_fb', 'dis_fb_x', 'dis_fb_y', 'frameId_ls',
       'frameId_bs', 'frames_bf_snap', 'initial_x_std', 'initial_y_std',
       'x_change_all', 'y_change_all', 'x_change', 'y_change', 'in_motion'],
      dtype='object')

In [8]:
#get initial y and final y coordinates (relative to the ball)
initial_rel_y_dis = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['dis_fb_y'].first().reset_index()
end_rel_y_dis = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['dis_fb_y'].last().reset_index()

df_player_play_motion = add_column_from_other_df(df_player_play_motion, initial_rel_y_dis, 'dis_fb_y', 'init_rel_y_dis')
df_player_play_motion = add_column_from_other_df(df_player_play_motion, end_rel_y_dis, 'dis_fb_y', 'end_rel_y_dis')

#calculate overall change in player's y coordinate
df_player_play_motion['overall_y_change'] = df_player_play_motion['init_rel_y_dis'] - df_player_play_motion['end_rel_y_dis']

#same steps but for x
initial_rel_x_dis = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['dis_fb_x'].first().reset_index()
end_rel_x_dis = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['dis_fb_x'].last().reset_index()

df_player_play_motion = add_column_from_other_df(df_player_play_motion, initial_rel_x_dis, 'dis_fb_x', 'init_rel_x_dis')
df_player_play_motion = add_column_from_other_df(df_player_play_motion, end_rel_x_dis, 'dis_fb_x', 'end_rel_x_dis')

df_player_play_motion['overall_x_change'] = (df_player_play_motion['init_rel_x_dis'] - df_player_play_motion['end_rel_x_dis'])

In [9]:
#find average player speed during their motion
motion_speed_avg = tracking_simple[tracking_simple['in_motion'] == 1].groupby(['gameId', 'playId', 'nflId'])['s_std'].mean().reset_index()
df_player_play_motion = add_column_from_other_df(df_player_play_motion, motion_speed_avg, 's_std', 'motion_s_avg')

In [10]:
#find player's total time in motion
motion_sum = tracking_simple.groupby(['gameId', 'playId', 'nflId'])['in_motion'].sum().reset_index()
df_player_play_motion = add_column_from_other_df(df_player_play_motion, motion_sum, 'in_motion', 'frames_in_motion')

In [11]:
tracking_simple[(tracking_simple['gameId'] == 2022090800)&(tracking_simple['playId'] == 80)&(tracking_simple['nflId'] == 47857.0)][:30]

,gameId,playId,nflId,displayName,frameId,club,event,x_std,y_std,o_std,...,frameId_ls,frameId_bs,frames_bf_snap,initial_x_std,initial_y_std,x_change_all,y_change_all,x_change,y_change,in_motion
96501,2022090800,80,47857.0,Devin Singletary,23,BUF,line_set,3826,8062,35506,...,23,88,65,3826,8062,0,0,0,0,0
96502,2022090800,80,47857.0,Devin Singletary,24,BUF,NaN,3826,8061,35564,...,23,88,64,3826,8062,0,1,0,-1,0
96503,2022090800,80,47857.0,Devin Singletary,25,BUF,NaN,3826,8061,35564,...,23,88,63,3826,8062,0,1,0,0,0
96504,2022090800,80,47857.0,Devin Singletary,26,BUF,NaN,3826,8061,35564,...,23,88,62,3826,8062,0,1,0,0,0
96505,2022090800,80,47857.0,Devin Singletary,27,BUF,NaN,3826,8061,35564,...,23,88,61,3826,8062,0,1,0,0,0
96506,2022090800,80,47857.0,Devin Singletary,28,BUF,NaN,3826,8061,35564,...,23,88,60,3826,8062,0,1,0,0,0
96507,2022090800,80,47857.0,Devin Singletary,29,BUF,NaN,3826,8061,35564,...,23,88,59,3826,8062,0,1,0,0,0
96508,2022090800,80,47857.0,Devin Singletary,30,BUF,NaN,3826,8061,35564,...,23,88,58,3826,8062,0,1,0,0,0
96509,2022090800,80,47857.0,Devin Singletary,31,BUF,NaN,3826,8061,35564,...,23,88,57,3826,8062,0,1,0,0,0
96510,2022090800,80,47857.0,Devin Singletary,32,BUF,NaN,3826,8059,35564,...,23,88,56,3826,8062,0,3,0,-2,0


In [12]:
# #Need some way to tell when a player switched direction

#get sign of first motion y frame
y_change_sign = tracking_simple[tracking_simple['in_motion'] == 1].groupby(['gameId', 'playId', 'nflId'])['y_change'].first().apply(lambda x: 1 if x > 0 else -1).reset_index(name='y_change_sign')

# Step 2: Merge initial sign back to the main dataframe
tracking_simple = tracking_simple.merge(
    y_change_sign, on=['gameId', 'playId', 'nflId'], how='left'
)


In [13]:
#column for if sign is different 
tracking_simple['y_sign_diff'] = (tracking_simple['y_change'] * tracking_simple['y_change_sign'] < 0)*1

#find earliest motion occurrence of the sign being different, return the y location
y_swap_loc = tracking_simple[tracking_simple['y_sign_diff'] == 1].groupby(['gameId', 'playId', 'nflId'])['dis_fb_y'].first().reset_index(name='y_change_sign')

#if none, get the location at the snap
y_swap_loc_fallback = (
    tracking_simple.groupby(['gameId', 'playId', 'nflId'])['dis_fb_y']
    .last()
    .reset_index(name='y_change_sign')
)

# Merge the two dataframes to ensure we get the y_change_sign values
y_swap_loc_merge = pd.merge(
    y_swap_loc_fallback, y_swap_loc,
    on=['gameId', 'playId', 'nflId'], how='left', suffixes=('_fallback', '_final')
)

# #add an additional boolean column to note if there was any switch that occurred on the play
y_swap_loc_merge['y_flip_bool'] = (
    (y_swap_loc_merge['y_change_sign_final'].notna()) & 
    (y_swap_loc_merge['y_change_sign_fallback'].sub(y_swap_loc_merge['y_change_sign_final']).abs() >= 100) #has to go at least a foot in other direction
).astype(int)

# If 'y_change_sign_final' is NaN, replace with 'y_change_sign_fallback'
y_swap_loc_merge['y_change_sign'] = y_swap_loc_merge['y_change_sign_final'].fillna(y_swap_loc_merge['y_change_sign_fallback'])

# Drop the extra columns
y_swap_loc_merge = y_swap_loc_merge[['gameId', 'playId', 'nflId', 'y_change_sign','y_flip_bool']]

In [14]:
#add the column to the dataframe
df_player_play_motion = add_column_from_other_df(df_player_play_motion, y_swap_loc_merge, 'y_change_sign', 'y_reverse_loc')
df_player_play_motion = add_column_from_other_df(df_player_play_motion, y_swap_loc_merge, 'y_flip_bool', 'y_reverse_bool')

C:\Users\etien\AppData\Local\Temp\ipykernel_2536\3041064243.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small.rename(columns={col: 'placeholder_column_x09ds623n'}, inplace=True)
C:\Users\etien\AppData\Local\Temp\ipykernel_2536\3041064243.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small.rename(columns={col: 'placeholder_column_x09ds623n'}, inplace=True)


In [15]:
#Convert inMotionAtBallSnap to numeric column
df_player_play_motion['inMotionAtBallSnap'] = df_player_play_motion['inMotionAtBallSnap'] * 1
df_player_play_motion['shiftSinceLineset'] = df_player_play_motion['shiftSinceLineset'] * 1
df_player_play_motion['motionSinceLineset'] = df_player_play_motion['motionSinceLineset'] * 1

In [16]:
# #column for if the player ever swapped sides of the football on the motion
# max_dist_fb = tracking_simple.groupby(['gameId','playId','nflId'])['y_dis_from_fb'].max().reset_index()
# min_dist_fb = tracking_simple.groupby(['gameId','playId','nflId'])['y_dis_from_fb'].min().reset_index()
# df_player_play_motion = add_column_from_other_df(df_player_play_motion, max_dist_fb, 'y_dis_from_fb', 'max_dist_fb')
# df_player_play_motion = add_column_from_other_df(df_player_play_motion, min_dist_fb, 'y_dis_from_fb', 'min_dist_fb')

# # Create a new column 'same_sign' based on the condition
# df_player_play_motion['passed_fb'] = ((df_player_play_motion['max_dist_fb'] >= 0) & (df_player_play_motion['min_dist_fb'] >= 0)) | \
#                                        ((df_player_play_motion['max_dist_fb'] < 0) & (df_player_play_motion['min_dist_fb'] < 0))
# df_player_play_motion['passed_fb'] = df_player_play_motion['passed_fb']*1

In [17]:
df_player.head()

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan


In [18]:
#create a column to see if a player changes direction on a play

#start by getting the sums of the y_change sign for every 
y_dis_travel_pos = tracking_simple[tracking_simple['y_change'] > 1].groupby(['gameId','playId','nflId'])['y_change'].sum().abs().reset_index()
y_dis_travel_neg = tracking_simple[tracking_simple['y_change'] < 1].groupby(['gameId','playId','nflId'])['y_change'].sum().abs().reset_index()

#add to df_player_play_motion
df_player_play_motion = add_column_from_other_df(df_player_play_motion, y_dis_travel_pos, 'y_change', 'y_dis_travel_pos')
df_player_play_motion = add_column_from_other_df(df_player_play_motion, y_dis_travel_neg, 'y_change', 'y_dis_travel_neg')

In [19]:
#if y started off negative, flip y values so it starts off positive (makes same type of motion on left or right side register the same)
mask = df_player_play_motion['init_rel_y_dis'] < 0
df_player_play_motion['y_flipped'] = mask.astype(int)
df_player_play_motion.loc[mask, ['init_rel_y_dis', 'end_rel_y_dis', 'overall_y_change','y_reverse_loc']] *= -1

#also swap 'y_dis_travel_pos' and 'y_dis_travel_neg' for rows where the mask is true
df_player_play_motion.loc[mask, ['y_dis_travel_pos', 'y_dis_travel_neg']] = \
    df_player_play_motion.loc[mask, ['y_dis_travel_neg', 'y_dis_travel_pos']].values

In [20]:
#bool if player swapped direction
ratio = df_player_play_motion['y_dis_travel_pos'] / df_player_play_motion['y_dis_travel_neg']
df_player_play_motion['swapped_directions'] = ((ratio >= 0.15) & (ratio <= 0.85)).astype(int)

In [21]:
# #column for their position
# df_positions = df_player[['nflId','position']]
# df_player_play_motion = df_player_play_motion.merge(df_positions, on=['nflId'], how='inner')

In [22]:
#note if player swapped sides of the field on the play
df_player_play_motion['swapped_side'] = (
    ((df_player_play_motion['init_rel_y_dis'] * df_player_play_motion['end_rel_y_dis'] < 0) &
     (df_player_play_motion['overall_y_change'] >= 150)) #ignore small fidgiting as swapping sides
    .astype(int)
)

In [23]:
#need to in tracking find relative distance from the football at every frame
# df_player_play_motion['dis_fb_x_max'] = tracking_simple.groupby(['gameId','playId','nflId'])['dis_fb_x'].max().reset_index()
# df_player_play_motion['dis_fb_x_min'] = tracking_simple.groupby(['gameId','playId','nflId'])['dis_fb_x'].min().reset_index()

# df_player_play_motion['dis_fb_y_max'] = tracking_simple.groupby(['gameId','playId','nflId'])['dis_fb_y'].max().reset_index()
# df_player_play_motion['dis_fb_y_min'] = tracking_simple.groupby(['gameId','playId','nflId'])['dis_fb_y'].min().reset_index()

#then find if for a player the min/max are both very far from the space where the player ends up

#and then give the coordinate of where the min/max is

In [24]:
tracking_simple.head()

,gameId,playId,nflId,displayName,frameId,club,event,x_std,y_std,o_std,...,frames_bf_snap,initial_x_std,initial_y_std,x_change_all,y_change_all,x_change,y_change,in_motion,y_change_sign,y_sign_diff
0,2022091200,64,46189.0,Will Dissly,43,SEA,line_set,3472,2646,443,...,71,3472,2646,0,0,0,0,0,-1.0,0
1,2022091200,64,46189.0,Will Dissly,44,SEA,NaN,3472,2646,565,...,70,3472,2646,0,0,0,0,0,-1.0,0
2,2022091200,64,46189.0,Will Dissly,45,SEA,NaN,3472,2646,709,...,69,3472,2646,0,0,0,0,0,-1.0,0
3,2022091200,64,46189.0,Will Dissly,46,SEA,NaN,3474,2646,831,...,68,3472,2646,-2,0,2,0,0,-1.0,0
4,2022091200,64,46189.0,Will Dissly,47,SEA,NaN,3475,2645,1172,...,67,3472,2646,-3,1,1,-1,0,-1.0,0


In [25]:
df_clustering_cols = df_player_play_motion[[
    'shiftSinceLineset', 'motionSinceLineset', 'snap_motion_gap', 
    'init_rel_y_dis', 'end_rel_y_dis', 'overall_y_change', 
    'init_rel_x_dis', 'end_rel_x_dis', 'overall_x_change', 
    'motion_s_avg', 'frames_in_motion', 'swapped_side',
    'y_dis_travel_pos', 'y_dis_travel_neg'
]]

In [26]:
#export dataframes
df_clustering_cols.to_csv('clustering_columns.csv', index=False)
df_player_play_motion.to_csv('df_player_play_motion.csv',index=False)